In [3]:
# ПРИХИ
import numpy as np
import pandas as pd

# Загружаем таблицу с приходами по всем ТГ
cs_prih = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\4001.xlsx',
    usecols=[0,4,9,11,12,15],names=['Производитель','Тип 1','Номер','Приходы_4001','ТПГ','ОПТ'], skiprows=4)

# Убираем "итог", оптовый товар и нулевые приходы

cs_prih.drop(index=len(cs_prih.Производитель.tolist())-1,inplace=True)

cs_prih = cs_prih[(cs_prih['ОПТ'] == 'Нет') & (cs_prih['Приходы_4001'] != 0)]
del cs_prih['ОПТ']

# Создаем 2 датафрейма с приходами ИФ и БК
prih_if = cs_prih[cs_prih['ТПГ']< 1499]

prih_bk = cs_prih[(cs_prih['ТПГ'] > 1499) &
                  (cs_prih['Тип 1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ')& 
                  (cs_prih['Тип 1'] != 'СЕРЬГИ-КОНГО')]

del prih_if['ТПГ'], prih_bk['ТПГ'], cs_prih['ТПГ']


In [2]:
# Дефицит и цена ИФ для печаток тоже
import os

def getfiles(dirpath):
    a = [s for s in os.listdir(dirpath) if '~$' not in s]
    a.sort(key=lambda s: os.path.getmtime(os.path.join(dirpath, s)))
    return os.path.join(dirpath, a[-1])

mydir = r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Карпова Ксения\ИФ\Дефицит'

deficit_df_if = pd.read_excel(getfiles(mydir),sheet_name='ЛЕНИЗ', 
                              skiprows=2, usecols=[1,19],names=['tpg','deficit'],nrows=102)

deficit_dict_if = deficit_df_if.set_index('tpg').T.to_dict('records')[0]

price_if = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\внав\заказ_и_цены\ИФ.xlsx',
                         usecols=['Номер','цена'], sheet_name='Лист1')
price_if['цена'] = price_if.цена.round(2)

In [14]:
#ИФ

# Загружаем две таблицы остатки и продажи
balance_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост',skiprows=9, usecols=[0, 1, 2, 3, 6, 7, 8, 9, 10])

balance_df.fillna(0, inplace=True)
balance_df['sum'] = balance_df.iloc[:, 3:].sum(axis=1)#удаляем строки с остатком 0 за все месяцы
balance = balance_df.set_index('sum').drop(0)
sales = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='прод',skiprows=12,usecols=list(range(2,9)))

# Притягиваем к остаткам продажи, потом добавляем к этому приходы через 'outer'
merged = pd.merge(prih_if,(pd.merge(balance,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'),  validate='one_to_one')),how = 'outer',  validate='one_to_one')

# Добавляем в финальную таблицу недостающую инфу по карточкам
items = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='карточки',skiprows=6)
                  
final = pd.merge(merged,items,on='Номер',how = 'left',  validate='one_to_one')

# Удаляем опт по группе наценки, ИМ по артикулу
final = final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]#na=False
final = final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]#na=False

# Добавляем дефицит по тпг и цену
final['Дефицит по тпг'] = final['Товарная подгруппа'].map(deficit_dict_if)
final = final.merge(price_if,on='Номер',how = 'left', validate='one_to_one')

#Удаляем буквы и зодиаки
final = final[~final['Дизайн'].isin(['ИФ БУКВЫ', 'ИФ ЗОДИАК', 'ИФ БУДДА'])]

# Выстраиваем столбцы в нужном порядке и сохраняем
col_names = final.columns.tolist()

col_rings = (['Артикул товара', 'Тип 1', 'Производитель', 'Товарная подгруппа'] +
           ['Ценовая корзина', 'Номер', 'Описание', 'Дизайн', 'Размер','Количество камней'] +
           col_names[10:16] + col_names[4:10] +
           ['Приходы_4001', 'Средний вес изделия'] +
           ['Дефицит по тпг','Тип 3','Вставка камней', 'Фото изделия','цена'])

col_norings = (['Артикул товара', 'Тип 1', 'Производитель', 'Товарная подгруппа'] +
           ['Ценовая корзина', 'Номер', 'Описание', 'Дизайн', 'Описание 3','Количество камней'] +
           col_names[10:16] + col_names[4:10] +
           ['Приходы_4001', 'Средний вес изделия'] +
           ['Дефицит по тпг', 'Тип 3','Вставка камней', 'Фото изделия', 'цена'])

rings = final[col_rings][final['Тип 1'].str.contains('КОЛЬЦО')]
no_rings = final[col_norings][~final['Тип 1'].str.contains('КОЛЬЦО')]

path = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Семен/заказы/'

rings.to_excel(path + 'иф_кольца.xlsx', index=False)
no_rings.to_excel(path + 'иф_некольца.xlsx', index=False)

In [15]:
neg_ring = rings[col_names[4:10]].lt(0).sum().to_frame('rings')
neg_noring = no_rings[col_names[4:10]].lt(0).sum().to_frame('norings')
neg_ring.join(neg_noring)

,rings,norings
"Сентябрь,2021_ос",0,0
"Октябрь,2021_ос",1,0
"Ноябрь,2021_ос",2,0
"Декабрь,2021_ос",4,0
"Январь,2022_ос",4,0
"Февраль,2022_ос",5,0


In [4]:
#БК 

# Загружаем остатки, продажи и карточки
balance_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='ост',skiprows=9, usecols=[0, 1, 2, 3, 6, 7, 8, 9, 10])

balance_df.fillna(0, inplace=True)
balance_df['sum'] = balance_df.iloc[:, 3:].sum(axis=1)#удаляем строки с остатком 0 за все месяцы
balance = balance_df.set_index('sum').drop(0)

sales = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='прод',skiprows=13, usecols=list(range(2,9)))

items = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='карточки',skiprows=6)

# Соединяем остатки и продажи, добавляем приходы через 'outer', добавляем инфу из карточек
merged = pd.merge(prih_bk,(pd.merge(balance,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'), validate='one_to_one')),how = 'outer', validate='one_to_one')
final = pd.merge(merged,items,on='Номер',how = 'left', validate='one_to_one')

# Удаляем опт, ИМ
final = final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]#na=False
final = final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]#na=False

# Создаем словарь с дефицитом по тпг и добавляем данные в таблицу
deficit_df_bk = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Ксю\Дефицит по ТПГ 2022.xlsx',
    sheet_name='Дефицит', skiprows=2, usecols=[0,18],names=['tpg','deficit'],nrows=97)

deficit_df_bk['tpg'] = deficit_df_bk.tpg.astype('float64')
deficit_dict_bk = dict(zip(deficit_df_bk.tpg, deficit_df_bk.deficit))
final['дефицит по тпг'] = final['Товарная подгруппа'].map(deficit_dict_bk)

# Добавляем информацию по цене
price_bk = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Назарова А.С.\Цены БК.xlsx',
    sheet_name='Нов', usecols=['Номер','цена']) 

final = final.merge(price_bk, on='Номер', how='left', validate='one_to_one')

# Выстраиваем столбцы в нужном порядке
ostatki_prodazi = final.columns.tolist()[10:16] + final.columns.tolist()[4:10]

columns = ['Артикул товара', 'Тип 1', 'Производитель', 'Товарная подгруппа', 'Ценовая корзина', 'Номер', 'Описание', 
           'Дизайн', 'Размер', 'Описание 3', *ostatki_prodazi, 'Приходы_4001','Средний вес изделия', 'дефицит по тпг', 
           'Тип 3', 'Фото изделия', 'цена', 'Тип изготовления']

final = final[columns]

#final=final.set_index('Артикул товара',drop=True, inplace=True)

# Фильтруем по кольцам убираем описание 3 и суммируем приходы 4001+4093 в "приходы". 
final_ring = final[(final['Тип 1'] =='КОЛЬЦО') | (final['Тип 1'] =='КОЛЬЦО ПЕЧАТКА')].drop(['Описание 3', 'Тип изготовления'], axis=1)


# Фильтруем по некольцам и убираем размер и буквы с зодиаками
final_other = final[~final['Тип 1'].str.contains('КОЛЬЦО')].drop(['Размер'], axis=1)

final_other = final_other[~(
    ((final_other['Тип 1'] == 'ПОДВЕС ДЕКОРАТИВНЫЙ') & 
     ((final_other['Дизайн'] == 'ИФ ДВУСПЛАВ ДЕКОР') | 
      (final_other['Дизайн'] =='ИФ БУКВЫ') | 
      (final_other['Дизайн'] =='ИФ ЗОДИАК'))) | 
    (final_other['Тип 1'] == 'СЕРЬГИ-КОНГО') | 
    (final_other['Тип 1'] == 'МОНЕТА')
    
)]

# сохраняем кольца и некольца на отдельные вкладки
with pd.ExcelWriter (
     r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\заказы_бк.xlsx'
) as writer:
    final_ring.to_excel(writer, sheet_name='кольца', index=False)
    final_other.to_excel(writer, sheet_name='некольца', index=False)


In [11]:
#ПЕЧАТКИ

# Загружаем остатки, продажи, карточки
path = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Семен/заказы/печатки.xlsx'

balance = pd.read_excel(path, sheet_name='ост_к',skiprows=9, usecols=list(range(9)))
sales = pd.read_excel(path, sheet_name='прод',skiprows=12,usecols=list(range(2,9)))
items = pd.read_excel(path, sheet_name='карточки',skiprows=6)

# Соединяем остатки и продажи, добавляем приходы через 'outer', добавляем инфу из карточек
merged = pd.merge(cs_prih[cs_prih['Тип 1'] == 'КОЛЬЦО ПЕЧАТКА'],(pd.merge(balance,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))),how = 'outer')
final = pd.merge(merged,items,on='Номер',how = 'left')

# Удаляем опт, ИМ
final = final[~(final['Группа наценки'].str.contains('опт',case=False,na=False)) & 
              ~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]#na=False

# Создаем таблицу с дефицитом по тпг и добавляем данные в основную таблицу
deficit_df_bk = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Ксю\Дефицит по ТПГ 2022.xlsx',
    sheet_name='Дефицит', skiprows=2, usecols=[0,18],names=['tpg','deficit'],nrows=97)

pech_if = deficit_df_if[deficit_df_if['tpg'].isin([1028,1029,1030])]
pech_bk = deficit_df_bk[deficit_df_bk['tpg'].isin([1538,1539,1540])]

deficit_pech = pd.concat([pech_if, pech_bk], ignore_index=True)
deficit_pech.rename(columns={'tpg':'Товарная подгруппа', 'deficit':'Дефицит по тпг'},inplace=True)

final = final.merge(deficit_pech, on='Товарная подгруппа',how = 'left')

# Добавляем цены
price_bk = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Назарова А.С.\Цены БК.xlsx',
    sheet_name='Нов', usecols=['Номер','цена']) 
price_bk.rename(columns={'Цена золото':'цена'}, inplace=True)

price_pech = pd.concat([price_bk, price_if], ignore_index=True)
final = final.merge(price_pech, on='Номер',how = 'left')

# Выстраиваем столбцы в нужном порядке и сохраняем
col_names = final.columns.tolist()

columns = (
    ['Артикул товара', 'Тип 1', 'Производитель', 'Товарная подгруппа', 'Ценовая корзина', 'Номер', 'Описание', 'Дизайн', 
     'Размер','Количество камней', *col_names[10:16], *col_names[4:10], 'Приходы_4001', 'Средний вес изделия', 
     'Дефицит по тпг','Тип 3','Вставка камней', 'Фото изделия','цена'])

final = final[columns]
#final=final.set_index('Артикул товара',drop=True, inplace=True)

final.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\заказ_печатки.xlsx', 
    index=False)


In [3]:

# ИФ РАЗДЕЛИТЬ ЗАКАЗЫ
import numpy as np
import pandas as pd
import re

# Правильно указать название столбца с заказом
path = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Семен/заказы/внав/заказ_и_цены/заказ.xlsx'
path1 = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Семен/заказы/внав/заказ_и_цены/ИФ.xlsx'
cols = ['Номер','Заказ итог','Средний вес изделия','Производитель','Дизайн','цена']
#cols = ['Номер','Заказ итог','Средний вес изделия','Производитель','Дизайн']# для буков
order = pd.read_excel(path, skiprows=1, sheet_name='заказ', usecols=cols)
order = order[order['Заказ итог'] > 0]
#order = order.merge(pd.read_excel(path1, sheet_name='Лист1', usecols=['Номер','цена']), on ='Номер', how='left')# для буков

order.rename(columns={'Производитель':'manuf', 'Номер':'tov', 'Дизайн':'design',
                      'Средний вес изделия':'weight', 'Заказ итог':'order','цена':'price'}, inplace=True)



#создаем словарь с категориями, по которым будут отдельные заказы по постам

category_dict = {
    'ИФ СУРА':'_mus', 'ИФ ПОЛУМЕСЯЦ':'_mus', 'ИФ МЕЧЕТЬ':'_mus', 
    "ИФ БЕЛОЕ ПУССЕТЫ": "_белое", "ИФ БЕЛОЕ": "_белое", "ИФ БЕЛОЕ ПУССЕТЫ ЦВЕТН": "_белое",
    "ИФ БЕЛОЕ ЦВЕТН": "_белое", "ИФ БЕЛОЕ ОБРУЧ": "_белое",
    "ИФ БУКВЫ": "_букв", "ИФ ЗОДИАК": "_зодиак"
}
order['design'] = order.design.apply(lambda x: category_dict.get(x) if x in list(category_dict.keys()) else '')
order['manuf'] = order['manuf'] + order['design']

# добавляем пустые столбцы
order['emp1'] = pd.Series()
order['emp2'] = pd.Series()

# убираем лишние знаки в названии поста 
order['manuf'] = order['manuf'].apply(lambda x: re.sub('''[ ?.!/;:'<>"«»]''','',x) ) 

# удаляем старые ексель файлы из папки
import os, glob
my_path = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Семен/заказы/внав/*.xlsx'
files = glob.glob(os.path.join(my_path))

for file in files:
    os.remove(file)
    
# сохраняем нарезая, убираем названия столбцов, индекс и колонку с производителем
directory = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Семен/заказы/внав/'
manuf_list = order['manuf'].unique().tolist()

for elem in manuf_list:    
    order[order['manuf'] == elem].to_excel(directory + elem + '.xlsx', index=False, header=False, 
                                           columns=['tov', 'emp1','order','emp2','weight', 'price'])


C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [29]:
mystr = r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен'
mystr.replace('\\','/')

'//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Семен'

In [33]:
# БК
import numpy as np
import pandas as pd
import re
# Правильно указать название столбца с заказом
order = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\внав\заказ_и_цены\заказ.xlsx', 
    skiprows=1, sheet_name='заказ', 
    usecols=['Номер', 'Заказ Ксю', 'Средний вес изделия', 'Производитель', 'цена'])

order.columns = ['manuf', 'tov', 'weight', 'order', 'price']

order = order[order['order'] > 0]

# отделяем заказы по определенным дизайнам
#mus_list = ['ИФ СУРА','ИФ ПОЛУМЕСЯЦ','ИФ МЕЧЕТЬ']

#order['manuf'].apply(lambda x: x+mus )

ValueError: Usecols do not match columns, columns expected but not found: ['цена']

In [6]:
# добавляем пустые столбцы
order['emp1'] = pd.Series()
order['emp2'] = pd.Series()

# порядок столбцов, убираем лишние знаки в названии поста 
order = order[['manuf', 'tov', 'emp1','order','emp2','weight', 'price']]
order['manuf'] = order['manuf'].apply(lambda x: re.sub('''[ ?.!/;:'<>"«»]''','',x) ) 

directory = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Семен/заказы/внав/'

# сохраняем нарезая, убираем названия столбцов, индекс и колонку с производителем
manuf_list = order['manuf'].unique().tolist()

for elem in manuf_list:    
    order[order['manuf'] == elem].to_excel(directory + elem + '.xlsx', index=False, header=False, 
                                           columns=['tov', 'emp1','order','emp2','weight', 'price'])

In [1]:
import pandas as pd
import numpy as np
#ДЕФИЦИТ ПО ГЛАДКОЙ ОБРУЧАЛКЕ НУЖНЫ ОСТАТКИ И НОРМЫ ПО РАЗМЕРАМ
# Загружаем остатки и нормы
stock = pd.read_excel(r'C:\Остатки\Обручальные кольца.xlsx', usecols=['КодСклада','ТоварнаяПодгруппа','РазмерИзделия'])
stock.rename(columns={'ТоварнаяПодгруппа': 'Товарная подгруппа', 'РазмерИзделия': 'Размер'}, inplace=True)
n_link =  r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx'

norms = pd.read_excel(n_link, sheet_name='гладкая_плоская размеры_тпг', skiprows=3)
del norms['адрес']
del norms['норма']

# выбрать остатки 1517-1524 тпг (без 1521-6,8)
stock = stock[(stock['Товарная подгруппа'] > 1516) & 
              (stock['Товарная подгруппа'] < 1525) & 
              (stock['Товарная подгруппа'] != 1521)]

# убрать нули в тпг и размерах и точку поменять на запятую
stock['Товарная подгруппа'] = stock['Товарная подгруппа'].astype('int')
# stock['Размер'] = stock['Размер'].apply(lambda size: str(int(size)) if size == int(size) else str(size))
# stock['Размер'] = stock['Размер'].apply(lambda size: size.replace('.',','))

# добавим в остатки колонку тпг_размер, группируем по этой колонке и считаем
stock[['Товарная подгруппа','Размер']] = stock[['Товарная подгруппа','Размер']].astype('str')
stock['size_tpg'] = stock['Товарная подгруппа'].str.cat(stock['Размер'], sep='_')
stock = stock.groupby(['КодСклада','size_tpg'])['size_tpg'].count().unstack().reset_index()

# Берем столбец с магами из норм и соединяем с остатками
stock.rename(columns={'КодСклада':'номер'},inplace=True)
stock = norms[['номер']].merge(stock, how='left')

# добавим недостающие колонки в остатки, чтобы получить идентичный нормам датафрейм
zero_cols = list(set(norms.columns) - set(stock.columns))

for cols in zero_cols:
    stock[cols] = np.nan
    
stock = stock[norms.columns.to_list()].fillna(0)

# Словарь соответствия тпг-дизайн+толщина
design_dict = {'1517':'ИФ ОБРУЧ ГЛАДКАЯ2', '1518':'ИФ ОБРУЧ ГЛАДКАЯ3','1519':'ИФ ОБРУЧ ГЛАДКАЯ4','1520':'ИФ ОБРУЧ ГЛАДКАЯ5',
               '1521':'ИФ ОБРУЧ ГЛАДКАЯ6','1522':'ИФ ОБРУЧ ПЛОСКАЯ3','1523':'ИФ ОБРУЧ ПЛОСКАЯ4','1524':'ИФ ОБРУЧ ПЛОСКАЯ5'}


# Вычитаем один датафрейм из другого, отрицательные значения меняем на нули, считаем сумму по строкам и сохраняем
if all(norms.columns == stock.columns):
    demand = norms - stock
    demand = demand.applymap(lambda num: 0 if num < 0 else num)
    demand = demand.T.iloc[1:]
    demand['sum'] = demand.sum(axis=1)
    demand.reset_index(inplace=True)
    demand['index'] = demand['index'].apply(lambda x: design_dict[x[:4]] + x[4:])
    demand[['index','sum']].to_excel('C:\Остатки\деф_обручи.xlsx', index=False)
else:
    print('столбцы не совпадают')    

In [12]:
dd
stock

,номер,1517_15,"1517_15,5",1517_16,"1517_16,5",1517_17,"1517_17,5",1517_18,"1517_18,5",1517_19,...,"1524_19,5",1524_20,"1524_20,5",1524_21,"1524_21,5",1524_22,"1524_22,5",1524_23,"1524_23,5",1524_24
0,4345,2.0,1.0,1.0,1.0,4.0,3.0,2.0,2.0,3.0,...,1.0,1.0,2.0,3.0,1.0,1.0,2.0,1.0,0.0,0.0
1,5155,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,...,0.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,1.0,0.0
2,4901,0.0,1.0,1.0,1.0,2.0,0.0,1.0,1.0,2.0,...,1.0,3.0,3.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0
3,5031,1.0,1.0,2.0,0.0,3.0,2.0,2.0,0.0,3.0,...,0.0,3.0,3.0,3.0,1.0,2.0,0.0,1.0,1.0,0.0
4,4992,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,...,1.0,2.0,3.0,2.0,2.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,4801,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,...,3.0,3.0,3.0,3.0,2.0,2.0,1.0,1.0,1.0,0.0
946,5385,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0
947,5430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
948,5432,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
import os
import pandas as pd


def read_and_save(file_name):
    
    path = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Назарова А.С/бланки_заказов/'
    path_read = path + file_name
    path_save = path + 'готовые/' + file_name
    df = pd.read_excel(path_read, skiprows=16, skipfooter=1)
    df = df.iloc[3:]

    #transpose df with sizes as columns
    sizes = df.set_index(['Артикул', 'Размер'])['Кол-во, шт.'].unstack()
    sizes.reset_index(inplace=True)

    # groupby df on article, add columns and regroup as needed
    group_cols = ['Артикул', 'Наименование', 'Проба', 'цвет металла', '%угара по металлу', 'Цена работы руб./гр.' ]
    df_grouped = df.groupby(group_cols)[['Кол-во, шт.','Общий вес, гр.']].sum()
    df_grouped.reset_index(inplace=True)
    df_grouped['Ср. вес., гр.'] = df_grouped['Общий вес, гр.']/df_grouped['Кол-во, шт.']
    df_grouped = df_grouped[df_grouped.columns[[0, 1, 2, 3, 4, 6, 8, 7, 5]]]

    df_grouped.merge(sizes, on='Артикул').to_excel(path_save, index=False)
    

directory = '//gold585.int/uk/Общее хранилище файлов/Коммерческий департамент/Отдел закупки/ЛИЧНЫЕ/Назарова А.С/бланки_заказов'

# delete old files
# for x in os.listdir(directory + '/готовые'):
#     os.remove(directory + '/готовые/' + x)
    
# create new files    
for x in os.listdir(directory):
    if 'xls' in x:
        read_and_save(x)    



In [7]:
file = pd.ExcelFile(r'C:\Users\Dotsenko.Semen\Documents\123.xlsx')
names = file.sheet_names

In [8]:
names

['Крест',
 'Образок',
 'Подвесы',
 'Бусины',
 'Цепи',
 'Шнурки кожаныетекстиль',
 'Кольца',
 'Браслеты',
 'Серьги']

In [18]:
pd.read_excel(r'C:\Users\Dotsenko.Semen\Documents\123.xlsx',sheet_name=names, usecols=['ШК']).values

<function dict.values>

In [9]:
emp = pd.DataFrame(columns=['Наименование', 'ШК'])
for name in names:
    emp.append(pd.read_excel(r'C:\Users\Dotsenko.Semen\Documents\123.xlsx',sheet_name=name, usecols=['Наименование', 'ШК']))
    

ValueError: Usecols do not match columns, columns expected but not found: ['Наименование']

In [24]:
pd.concat(pd.read_excel(r'C:\Users\Dotsenko.Semen\Documents\123.xlsx', sheet_name=None), ignore_index=True).to_excel(r'C:\Users\Dotsenko.Semen\Documents\12453.xlsx')
